In [ ]:
import shapely.geometry
import shapely.affinity
import skimage.measure
import skimage.draw
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
t = np.linspace(0, 2 * np.pi, endpoint=False)
x = 1.5 * np.sin(t)
y = np.cos(t) + 0.5

In [ ]:
plt.plot(x, y)
plt.axis('equal')

In [ ]:
def grid_to_transform(grid):
    x0 = grid[0]
    all_dx = np.diff(grid)
    dx = all_dx[0]
    assert np.allclose(dx, all_dx)
    
    return x0, dx

In [ ]:
def mask_to_contours(x_grid, y_grid, mask):
    padded_mask = np.pad(mask, 1)
    contours_coords_padded_image_frame = skimage.measure.find_contours(padded_mask, level=127.5)
    contours_coords_image_frame = [item -1 for item in contours_coords_padded_image_frame]
    
    x0, dx = grid_to_transform(x_grid)
    y0, dy = grid_to_transform(y_grid)

    contours = []
    for yx_coords in contours_coords_image_frame:
        yx_coords[:, 1] = yx_coords[:, 1] * dx + x0
        yx_coords[:, 0] = yx_coords[:, 0] * dy + y0
        
        contours.append(yx_coords)
        
    return contours

In [ ]:
def _contours_to_expanded_mask(x_grid, y_grid, contours, expansion):
    mask_size = (len(y_grid), len(x_grid))
    expanded_mask_size = np.array(mask_size) * expansion
    
    x0, dx = grid_to_transform(x_grid)
    y0, dy = grid_to_transform(y_grid)
    
    expanded_mask = np.zeros(expanded_mask_size)
    
    for yx_coords in contours:
        y = yx_coords[:, 0]
        x = yx_coords[:, 1]
        
        i = ((y - y0) / dy) * expansion + (expansion - 1) * 0.5
        j = ((x - x0) / dx) * expansion + (expansion - 1) * 0.5
        
        expanded_mask = np.logical_or(
            expanded_mask,
            skimage.draw.polygon2mask(expanded_mask_size, np.array(list(zip(i, j)))),
        )
        
    return expanded_mask

In [ ]:
def contours_to_mask(x_grid, y_grid, contours, expansion=16):
    expanded_mask = _contours_to_expanded_mask(x_grid, y_grid, contours, expansion)
    float_mask = skimage.measure.block_reduce(expanded_mask, block_size=(expansion, expansion), func=np.mean)
    
    mask = np.round(float_mask * 255).astype(np.uint8)
    
    return mask

In [ ]:
yx_coords = np.concatenate([y[:, None], x[:, None]], axis=-1)
yx_coords.shape

In [ ]:
contours = [yx_coords]

In [ ]:
x_grid = np.linspace(-2, 2, 21)
y_grid = np.linspace(-2, 2, 31)

In [ ]:
mask = contours_to_mask(x_grid, y_grid, contours)

In [ ]:
round_trip_contours = mask_to_contours(x_grid, y_grid, mask)
rount_trip_y = round_trip_contours[0][:, 0]
rount_trip_x = round_trip_contours[0][:, 1]

In [ ]:
plt.pcolormesh(x_grid, y_grid, mask, shading="nearest")
plt.plot(x, y)
plt.plot(rount_trip_x, rount_trip_y)

plt.axis('equal')